# Part B - Neural Style Transfer: Recreating Van Gogh's Style

## Deep Learning Project - Tel Aviv University

This notebook implements:
- **Part 2A**: Style transfer function (from Nir_part_B)
- **Part 2B**: Hyperparameter search using Optuna to find optimal style transfer parameters
- **Part 2C**: Application to 20 images using both VGG-19 and AlexNet, with evaluation

### Overview:
1. **Setup**: Load Part A classifiers and prepare style transfer components
2. **Part 2A**: Style transfer function implementation
3. **Part 2B**: Optuna search for optimal hyperparameters (content_weight, style_weight, etc.)
4. **Part 2C**: Apply style transfer to 20 images, evaluate with both classifiers

### Requirements:
- Part A trained models (best_vangogh_classifier.pth)
- Van Gogh paintings as style images
- Content images (personal photos or other images)


## 1. Environment Setup


In [ ]:
# Check if running on Kaggle
import os
import sys

IN_KAGGLE = os.path.exists('/kaggle')
IN_COLAB = 'google.colab' in sys.modules

if IN_KAGGLE:
    print("Running on Kaggle")
elif IN_COLAB:
    print("Running on Google Colab")
else:
    print("Running locally")


In [ ]:
# Install required packages
%pip install -q optuna wandb


## 2. Import Libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import pickle
import time

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, models

# Hyperparameter tuning
import optuna
import wandb

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


## 3. Load Part A Classifiers


In [ ]:
# Load or Create Part A trained models
# These will serve as "judges" to evaluate style transfer quality

import os  # Ensure os is imported for path operations

# Model creation function from Part A
def create_model(model_name='VGG19', freeze_features=True, dropout=0.5):
    """
    Create model with binary classifier (from Part A).
    Implements Smart Fine-tuning: When freeze_features=False, unfreezes only top layers.
    
    Args:
        model_name: 'VGG19' or 'AlexNet'
        freeze_features: If True, freeze feature extractor (only train classifier)
                        If False, unfreeze only top layers (Smart Fine-tuning)
        dropout: Dropout rate for classifier (0.0 to 0.7)
    """
    if model_name == 'VGG19':
        model = models.vgg19(weights='IMAGENET1K_V1')
        # First, freeze all feature layers by default
        for param in model.features.parameters():
            param.requires_grad = False
        
        # Smart Fine-tuning: If freeze_features=False, unfreeze only the last 8 layers
        if not freeze_features:
            # VGG19 features has 36 layers (0-35), unfreeze last 8 layers (28-35)
            for i in range(28, 36):
                for param in model.features[i].parameters():
                    param.requires_grad = True
        
        # Modify classifier: VGG19 classifier[6] is the last Linear layer
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(num_features, 2)  # Binary classification
        )
    elif model_name == 'AlexNet':
        model = models.alexnet(weights='IMAGENET1K_V1')
        # First, freeze all feature layers by default
        for param in model.features.parameters():
            param.requires_grad = False
        
        # Smart Fine-tuning: If freeze_features=False, unfreeze only the last 2 layers
        if not freeze_features:
            # AlexNet features has 12 layers (0-11), unfreeze last 2 layers (10-11)
            for i in range(10, 12):
                for param in model.features[i].parameters():
                    param.requires_grad = True
        
        # Modify classifier: AlexNet classifier[6] is the last Linear layer
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(num_features, 2)  # Binary classification
        )
    else:
        raise ValueError(f"Unknown model type: {model_name}")
    
    return model.to(device)

def load_classifier(model_name='VGG19', checkpoint_path=None):
    """Try to load a trained classifier from Part A, or create a new one"""
    if checkpoint_path is None:
        # Try to find checkpoint file
        possible_paths = [
            '/kaggle/working/best_vangogh_classifier.pth',
            '/kaggle/input/part-a-models/best_vangogh_classifier.pth',
            'best_vangogh_classifier.pth'
        ]
        checkpoint_path = None
        for path in possible_paths:
            if os.path.exists(path):
                checkpoint_path = path
                break
    
    if checkpoint_path and os.path.exists(checkpoint_path):
        # Load from checkpoint
        print(f"Loading classifier from: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        best_params = checkpoint.get('best_params', {})
        
        # Create model based on saved parameters
        model_type = best_params.get('model_name', model_name)
        dropout = best_params.get('dropout', 0.5)
        freeze_features = best_params.get('freeze_features', True)
        
        model = create_model(model_name=model_type, freeze_features=freeze_features, dropout=dropout)
        
        # Load weights
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
        
        print(f"Loaded {model_type} classifier (Val Acc: {checkpoint.get('best_val_acc', 'N/A'):.4f})")
        return model, model_type
    else:
        # No checkpoint found - create a new model with default parameters
        print(f"No checkpoint found. Creating new {model_name} classifier with default parameters.")
        print("Note: This model will not be trained. For best results, train Part A first.")
        
        # Use reasonable default parameters (similar to Part A best params)
        if model_name == 'VGG19':
            model = create_model(model_name='VGG19', freeze_features=False, dropout=0.4)
        else:
            model = create_model(model_name='AlexNet', freeze_features=False, dropout=0.4)
        
        model.eval()
        print(f"Created {model_name} classifier (untrained - will use ImageNet features only)")
        return model, model_name

# Load the judge model (try to load from checkpoint, or create new)
# We'll use this as the judge for hyperparameter search
try:
    judge_model, judge_model_name = load_classifier()
    print(f"\nUsing {judge_model_name} as judge for hyperparameter search")
except Exception as e:
    print(f"Error: {e}")
    print("Creating default VGG19 classifier as fallback...")
    judge_model, judge_model_name = load_classifier('VGG19')


## 4. Part 2A: Style Transfer Function

This section implements the generic style transfer function (from Nir_part_B)


In [ ]:
# Image loading and preprocessing
imsize = 512 if torch.cuda.is_available() else 128

loader = transforms.Compose([
    transforms.Resize((imsize, imsize)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def image_loader(image_path):
    """Load and preprocess an image"""
    image = Image.open(image_path).convert('RGB')
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

def imshow(tensor, title=None, save_path=None):
    """Display a tensor as an image"""
    image = tensor.cpu().clone().detach().squeeze(0)
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    image = inv_normalize(image)
    image = transforms.ToPILImage()(image.clamp(0, 1))
    
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')
    
    if save_path:
        image.save(save_path)
    
    plt.show()
    return image


In [ ]:
# Helper function for Gram matrix (used in style loss)
def get_gram_matrix(tensor):
    """Compute Gram matrix for style loss"""
    b, c, h, w = tensor.size()
    features = tensor.view(b * c, h * w)
    gram = torch.mm(features, features.t())
    return gram / (b * c * h * w)


In [ ]:
# Function to extract features from model layers
def get_features(image, model, layers_dict):
    """Extract activation maps from selected layers in the model"""
    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers_dict:
            features[layers_dict[name]] = x
    return features


In [ ]:
def neural_style_transfer(model, content_image, style_image, 
                          content_layers, style_layers, 
                          content_weight, style_weight, 
                          style_layer_weights, num_steps=300):
    """
    Generic neural style transfer function
    
    Args:
        model: Pre-trained network (VGG-19 or AlexNet features)
        content_image: Image whose structure to preserve
        style_image: Image whose artistic style to apply
        content_layers: Dict mapping layer indices to names for content features
        style_layers: Dict mapping layer indices to names for style features
        content_weight: Scalar (α) to control emphasis on content
        style_weight: Scalar (β) to control intensity of style
        style_layer_weights: Dict of weights to balance each style layer contribution
        num_steps: Number of optimization steps
    
    Returns:
        Final stylized image tensor
    """
    # Set model to eval mode and freeze parameters
    model.eval()
    for param in model.parameters():
        param.requires_grad = False
    
    # Initialize target image as clone of content image
    target = content_image.clone().requires_grad_(True)
    
    # Combine all layers needed
    all_layers = {**content_layers, **style_layers}
    
    # Extract fixed features from source images
    content_features = get_features(content_image, model, content_layers)
    style_features = get_features(style_image, model, style_layers)
    
    # Compute Gram matrices for style image
    style_grams = {layer: get_gram_matrix(style_features[layer]) for layer in style_features}

    # Setup optimizer (LBFGS recommended for NST)
    optimizer = optim.LBFGS([target])
    
    # Optimization loop
    for i in range(num_steps):
        def closure():
            optimizer.zero_grad()
            
            # Extract current features from target
            target_features = get_features(target, model, all_layers)
            
            # Compute Content Loss
            c_loss = 0
            for layer in content_layers.values():
                c_loss += torch.mean((target_features[layer] - content_features[layer])**2)
            
            # Compute Style Loss
            s_loss = 0
            for layer in style_layers.values():
                target_gram = get_gram_matrix(target_features[layer])
                style_gram = style_grams[layer]
                layer_weight = style_layer_weights.get(layer, 1.0)
                s_loss += layer_weight * torch.mean((target_gram - style_gram)**2)
            
            # Total weighted loss
            total_loss = content_weight * c_loss + style_weight * s_loss
            total_loss.backward()
            
            return total_loss

        optimizer.step(closure)
        
        # Optional: print progress every 50 steps
        if (i + 1) % 50 == 0:
            print(f"Step {i+1}/{num_steps} completed")
    
    return target.detach()


## 5. Prepare Style and Content Images


In [ ]:
# Find Van Gogh paintings to use as style images
# We need at least 5 different Van Gogh paintings

def find_van_gogh_paintings(base_dir, min_count=5):
    """Find Van Gogh painting files"""
    van_gogh_files = []
    
    # Search for specific famous paintings
    keywords = [
        "starry-night",
        "sunflowers",
        "crows",  # Wheatfield with Crows
        "cafe-terrace",
        "irises",
        "bedroom",
        "almond-blossom",
        "self-portrait"
    ]
    
    for fname in os.listdir(base_dir):
        if not fname.lower().endswith((".jpg", ".png")):
            continue
        
        if "van-gogh" in fname.lower():
            # Check if it matches any keyword
            for keyword in keywords:
                if keyword in fname.lower():
                    filepath = os.path.join(base_dir, fname)
                    if filepath not in van_gogh_files:
                        van_gogh_files.append(filepath)
                    break
    
    # If we don't have enough, add more random Van Gogh paintings
    if len(van_gogh_files) < min_count:
        for fname in os.listdir(base_dir):
            if len(van_gogh_files) >= min_count:
                break
            if not fname.lower().endswith((".jpg", ".png")):
                continue
            if "van-gogh" in fname.lower():
                filepath = os.path.join(base_dir, fname)
                if filepath not in van_gogh_files:
                    van_gogh_files.append(filepath)
    
    return van_gogh_files[:min_count] if len(van_gogh_files) >= min_count else van_gogh_files

# Find base directory
possible_dirs = [
    "/kaggle/input/wikiart/Post_Impressionism",
    "/kaggle/input/Post_Impressionism",
    "/content/data/Post_Impressionism",
    "./Post_Impressionism"
]

base_dir = None
for dir_path in possible_dirs:
    if os.path.exists(dir_path):
        base_dir = dir_path
        break

if base_dir:
    style_image_paths = find_van_gogh_paintings(base_dir, min_count=5)
    print(f"Found {len(style_image_paths)} Van Gogh style images:")
    for i, path in enumerate(style_image_paths, 1):
        print(f"  {i}. {os.path.basename(path)}")
else:
    print("Warning: Could not find Post_Impressionism directory")
    print("Please manually specify style_image_paths below")
    style_image_paths = []


In [ ]:
# Manually specify style images if automatic search didn't work
# Or add your preferred Van Gogh paintings here
if len(style_image_paths) < 5:
    # Example paths (adjust based on your setup)
    style_image_paths = [
        "/kaggle/input/wikiart/Post_Impressionism/vincent-van-gogh_the-starry-night-1889(1).jpg",
        "/kaggle/input/wikiart/Post_Impressionism/vincent-van-gogh_sunflowers-1888.jpg",
        "/kaggle/input/wikiart/Post_Impressionism/vincent-van-gogh_wheatfield-with-crows-1890.jpg",
        "/kaggle/input/wikiart/Post_Impressionism/vincent-van-gogh_cafe-terrace-at-night-1888.jpg",
        "/kaggle/input/wikiart/Post_Impressionism/vincent-van-gogh_irises-1889.jpg"
    ]
    
    # Filter to only existing files
    style_image_paths = [p for p in style_image_paths if os.path.exists(p)]
    print(f"Using {len(style_image_paths)} manually specified style images")


In [ ]:
# Prepare content images
# You should provide 20 content images (personal photos are encouraged!)
# For now, we'll create a placeholder that you can replace

def find_content_images(content_dir=None, count=20):
    """Find content images for style transfer"""
    content_paths = []
    
    # Try to find content images directory
    possible_dirs = [
        "/kaggle/input/content",
        "/kaggle/input/content-images",
        "./content_images",
        content_dir
    ]
    
    content_base = None
    for dir_path in possible_dirs:
        if dir_path and os.path.exists(dir_path):
            content_base = dir_path
            break
    
    if content_base:
        # Load all images from directory
        for fname in os.listdir(content_base):
            if fname.lower().endswith((".jpg", ".jpeg", ".png")):
                content_paths.append(os.path.join(content_base, fname))
                if len(content_paths) >= count:
                    break
    
    return content_paths

content_image_paths = find_content_images(count=20)

if len(content_image_paths) < 20:
    print(f"Warning: Found only {len(content_image_paths)} content images")
    print("Please add more content images or use images from the dataset")
    
    # Fallback: use some non-Van-Gogh paintings as content images
    if base_dir:
        print("Using non-Van-Gogh paintings as content images...")
        for fname in os.listdir(base_dir):
            if len(content_image_paths) >= 20:
                break
            if not fname.lower().endswith((".jpg", ".png")):
                continue
            if "van-gogh" not in fname.lower():
                content_image_paths.append(os.path.join(base_dir, fname))

print(f"\nPrepared {len(content_image_paths)} content images")


## 6. Define Layer Configurations for VGG-19 and AlexNet


In [ ]:
# VGG-19 layer configuration
VGG19_CONTENT_LAYERS = {'21': 'content'}  # conv4_2

VGG19_STYLE_LAYERS = {
    '0': 'conv1_1',
    '5': 'conv2_1',
    '10': 'conv3_1',
    '19': 'conv4_1',
    '28': 'conv5_1'
}

VGG19_STYLE_LAYER_WEIGHTS = {
    'conv1_1': 1.0,
    'conv2_1': 0.8,
    'conv3_1': 0.5,
    'conv4_1': 0.3,
    'conv5_1': 0.1
}

# AlexNet layer configuration (simpler, fewer layers)
ALEXNET_CONTENT_LAYERS = {'8': 'content'}  # conv3

ALEXNET_STYLE_LAYERS = {
    '0': 'conv1',
    '3': 'conv2',
    '6': 'conv3'
}

ALEXNET_STYLE_LAYER_WEIGHTS = {
    'conv1': 1.0,
    'conv2': 0.75,
    'conv3': 0.5
}

print("Layer configurations defined for VGG-19 and AlexNet")


## 7. Part 2B: Hyperparameter Search with Optuna

Find optimal style transfer hyperparameters using the Part 1 classifier as a judge.


In [ ]:
# Login to Weights & Biases
wandb.login(key="16d1bc863b28f81253ac0ee253b453393791a7e1")
print("Logged in to Weights & Biases")


In [ ]:
# Load VGG-19 features model for style transfer
vgg_features = models.vgg19(weights='IMAGENET1K_V1').features.to(device).eval()
for param in vgg_features.parameters():
    param.requires_grad = False
print("VGG-19 features model loaded for style transfer")


In [ ]:
# Select fixed content and style images for hyperparameter search
# Using one content image and one style image for consistency
if len(content_image_paths) > 0 and len(style_image_paths) > 0:
    search_content_path = content_image_paths[0]
    search_style_path = style_image_paths[0]  # Use first Van Gogh painting
    
    search_content_img = image_loader(search_content_path)
    search_style_img = image_loader(search_style_path)
    
    print(f"Using content image: {os.path.basename(search_content_path)}")
    print(f"Using style image: {os.path.basename(search_style_path)}")
    
    # Display the images
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    imshow(search_content_img, "Content Image")
    plt.subplot(1, 2, 2)
    imshow(search_style_img, "Style Image (Van Gogh)")
else:
    raise ValueError("Need at least one content and one style image for hyperparameter search!")


In [ ]:
def style_transfer_objective(trial):
    """
    Optuna objective function for style transfer hyperparameter search.
    Maximizes the classifier's probability that the stylized image is Van Gogh.
    """
    # Suggest hyperparameters
    content_weight = trial.suggest_float("content_weight", 1e-4, 1e2, log=True)
    style_weight = trial.suggest_float("style_weight", 1e3, 1e9, log=True)
    
    # Optional: also search style_layer_weights (can be commented out for faster search)
    # For now, use fixed style_layer_weights to reduce search space
    
    # Run style transfer with suggested parameters
    # Use fewer steps for faster search (can increase for final results)
    num_steps = 100  # Reduced for faster hyperparameter search
    
    try:
        stylized_image = neural_style_transfer(
            vgg_features,
            search_content_img,
            search_style_img,
            VGG19_CONTENT_LAYERS,
            VGG19_STYLE_LAYERS,
            content_weight,
            style_weight,
            VGG19_STYLE_LAYER_WEIGHTS,
            num_steps=num_steps
        )
        
        # Prepare image for classifier (resize to 224x224)
        judge_input = F.interpolate(stylized_image, size=(224, 224), mode='bilinear', align_corners=False)
        
        # Get score from judge (classifier)
        with torch.no_grad():
            output = judge_model(judge_input)
            probabilities = F.softmax(output, dim=1)
            # Assuming class 1 is "Van Gogh" (check your model's class order)
            van_gogh_prob = probabilities[0][1].item()  # Probability of being Van Gogh
        
        return van_gogh_prob
    
    except Exception as e:
        print(f"Error in trial {trial.number}: {e}")
        return 0.0  # Return low score on error

print("Style transfer objective function defined")


In [ ]:
# Run Optuna hyperparameter search
print("="*60)
print("PART 2B: HYPERPARAMETER SEARCH FOR STYLE TRANSFER")
print("="*60)
print("\nObjective: Maximize classifier's Van Gogh probability")
print("This may take 30-60 minutes depending on number of trials...")

# Create study
study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=2)
)

# Initialize wandb run for tracking
wandb_run = wandb.init(
    project="VanGogh_StyleTransfer_HPO",
    name="style_transfer_hyperparameter_search",
    config={"judge_model": judge_model_name}
)

# Track start time
start_time = time.time()

# Run optimization (adjust n_trials and timeout as needed)
# For faster results, use fewer trials; for better results, use more
study.optimize(style_transfer_objective, n_trials=15, timeout=3600, show_progress_bar=True)

elapsed_time = time.time() - start_time
elapsed_minutes = elapsed_time / 60

print("\n" + "="*60)
print("HYPERPARAMETER SEARCH COMPLETE!")
print("="*60)
print(f"\nTime taken: {elapsed_minutes:.2f} minutes ({elapsed_time:.0f} seconds)")
print(f"Completed trials: {len(study.trials)}")
print(f"\nBest Van Gogh probability: {study.best_value:.4f}")
print(f"\nOptimal hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Log best parameters to wandb
wandb.log({"best_van_gogh_prob": study.best_value, **study.best_params})
wandb_run.finish()

# Save results
best_style_transfer_params = study.best_params
with open('best_style_transfer_params.pkl', 'wb') as f:
    pickle.dump(best_style_transfer_params, f)
print("\nSaved optimal parameters to 'best_style_transfer_params.pkl'")


## 8. Part 2C: Application & Evaluation

Apply style transfer to 20 images using both VGG-19 and AlexNet, then evaluate with both classifiers.


In [ ]:
# Load optimal hyperparameters (or use defaults if search wasn't run)
if 'best_style_transfer_params' not in globals():
    if os.path.exists('best_style_transfer_params.pkl'):
        with open('best_style_transfer_params.pkl', 'rb') as f:
            best_style_transfer_params = pickle.load(f)
        print(f"Loaded optimal parameters from file")
    else:
        # Use reasonable defaults if search wasn't run
        best_style_transfer_params = {
            'content_weight': 1e-2,
            'style_weight': 1e6
        }
        print("Using default parameters (hyperparameter search not run)")

print(f"\nUsing hyperparameters:")
for k, v in best_style_transfer_params.items():
    print(f"  {k}: {v}")


In [ ]:
# Load both VGG-19 and AlexNet feature models
vgg19_features = models.vgg19(weights='IMAGENET1K_V1').features.to(device).eval()
for param in vgg19_features.parameters():
    param.requires_grad = False

alexnet_features = models.alexnet(weights='IMAGENET1K_V1').features.to(device).eval()
for param in alexnet_features.parameters():
    param.requires_grad = False

print("Both VGG-19 and AlexNet feature models loaded")


In [ ]:
# Set consistent number of epochs/steps for all style transfers
# This must be the same for all images and models (project requirement)
CONSISTENT_NUM_STEPS = 300  # You can adjust this, but keep it consistent

print(f"Using {CONSISTENT_NUM_STEPS} optimization steps for all style transfers")


In [ ]:
# Function to apply style transfer and evaluate
def apply_and_evaluate_style_transfer(content_path, style_path, model_name='VGG19', 
                                     save_dir='/kaggle/working/stylized_images'):
    """
    Apply style transfer and evaluate with both classifiers
    
    Returns:
        dict with stylized image, evaluation scores, and metadata
    """
    # Load images
    content_img = image_loader(content_path)
    style_img = image_loader(style_path)
    
    # Select model and layer configuration
    if model_name == 'VGG19':
        model = vgg19_features
        content_layers = VGG19_CONTENT_LAYERS
        style_layers = VGG19_STYLE_LAYERS
        style_layer_weights = VGG19_STYLE_LAYER_WEIGHTS
    elif model_name == 'AlexNet':
        model = alexnet_features
        content_layers = ALEXNET_CONTENT_LAYERS
        style_layers = ALEXNET_STYLE_LAYERS
        style_layer_weights = ALEXNET_STYLE_LAYER_WEIGHTS
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    # Apply style transfer
    print(f"\nApplying {model_name} style transfer...")
    stylized = neural_style_transfer(
        model, content_img, style_img,
        content_layers, style_layers,
        best_style_transfer_params['content_weight'],
        best_style_transfer_params['style_weight'],
        style_layer_weights,
        num_steps=CONSISTENT_NUM_STEPS
    )
    
    # Prepare for classifier evaluation
    eval_input = F.interpolate(stylized, size=(224, 224), mode='bilinear', align_corners=False)
    
    # Evaluate with judge model (and potentially other classifiers)
    with torch.no_grad():
        output = judge_model(eval_input)
        probs = F.softmax(output, dim=1)
        judge_van_gogh_prob = probs[0][1].item()
        judge_prediction = output.argmax(dim=1).item()
    
    # Save stylized image
    os.makedirs(save_dir, exist_ok=True)
    content_name = os.path.splitext(os.path.basename(content_path))[0]
    style_name = os.path.splitext(os.path.basename(style_path))[0]
    save_path = os.path.join(save_dir, f"{model_name}_{content_name}_{style_name}.jpg")
    
    # Convert and save
    result_image = imshow(stylized, f"{model_name}: {os.path.basename(content_path)}", save_path=save_path)
    
    return {
        'stylized_image': stylized,
        'content_path': content_path,
        'style_path': style_path,
        'model_name': model_name,
        'judge_van_gogh_prob': judge_van_gogh_prob,
        'judge_prediction': judge_prediction,
        'save_path': save_path
    }

print("Style transfer application function defined")


In [ ]:
# Apply style transfer to 20 images with both VGG-19 and AlexNet
# Use at least 5 different Van Gogh paintings as style images

print("="*60)
print("PART 2C: APPLYING STYLE TRANSFER TO 20 IMAGES")
print("="*60)

# Ensure we have enough content and style images
num_content = min(20, len(content_image_paths))
num_styles = min(5, len(style_image_paths))

print(f"\nProcessing {num_content} content images with {num_styles} style images")
print(f"Using {CONSISTENT_NUM_STEPS} steps for all transfers")

# Store all results
all_results = []

# Process each content image with each style image, using both models
content_to_process = content_image_paths[:num_content]
styles_to_use = style_image_paths[:num_styles]

# Distribute style images across content images
# Each content image gets styled with different Van Gogh paintings
for i, content_path in enumerate(tqdm(content_to_process, desc="Processing content images")):
    # Cycle through style images
    style_idx = i % num_styles
    style_path = styles_to_use[style_idx]
    
    # Apply with VGG-19
    try:
        vgg_result = apply_and_evaluate_style_transfer(
            content_path, style_path, model_name='VGG19'
        )
        all_results.append(vgg_result)
    except Exception as e:
        print(f"Error with VGG-19 on {content_path}: {e}")
    
    # Apply with AlexNet
    try:
        alexnet_result = apply_and_evaluate_style_transfer(
            content_path, style_path, model_name='AlexNet'
        )
        all_results.append(alexnet_result)
    except Exception as e:
        print(f"Error with AlexNet on {content_path}: {e}")

print(f"\nCompleted {len(all_results)} style transfers")


In [ ]:
# Evaluate results with both classifiers
# Load both VGG-19 and AlexNet classifiers if available

def evaluate_with_classifier(stylized_image, classifier, classifier_name):
    """Evaluate a stylized image with a classifier"""
    eval_input = F.interpolate(stylized_image, size=(224, 224), mode='bilinear', align_corners=False)
    
    with torch.no_grad():
        output = classifier(eval_input)
        probs = F.softmax(output, dim=1)
        van_gogh_prob = probs[0][1].item()
        prediction = output.argmax(dim=1).item()
    
    return {
        f'{classifier_name}_van_gogh_prob': van_gogh_prob,
        f'{classifier_name}_prediction': prediction
    }

# Try to load both classifiers
vgg19_classifier = None
alexnet_classifier = None

try:
    # Try loading VGG-19 classifier
    vgg19_classifier, _ = load_classifier('VGG19')
except:
    print("Could not load separate VGG-19 classifier, using judge model")

try:
    # Try loading AlexNet classifier
    alexnet_classifier, _ = load_classifier('AlexNet')
except:
    print("Could not load separate AlexNet classifier, using judge model")

# Evaluate all results with available classifiers
print("\nEvaluating all stylized images with classifiers...")

for result in tqdm(all_results, desc="Evaluating"):
    stylized = result['stylized_image']
    
    # Evaluate with judge model (already done, but add to results)
    result['judge_eval'] = {
        'van_gogh_prob': result['judge_van_gogh_prob'],
        'prediction': result['judge_prediction']
    }
    
    # Evaluate with VGG-19 classifier if available
    if vgg19_classifier is not None:
        vgg_eval = evaluate_with_classifier(stylized, vgg19_classifier, 'VGG19_classifier')
        result.update(vgg_eval)
    
    # Evaluate with AlexNet classifier if available
    if alexnet_classifier is not None:
        alexnet_eval = evaluate_with_classifier(stylized, alexnet_classifier, 'AlexNet_classifier')
        result.update(alexnet_eval)

print("\nEvaluation complete!")


In [ ]:
# Analyze and summarize results
print("="*60)
print("PART 2C: RESULTS SUMMARY")
print("="*60)

# Convert results to DataFrame for analysis
results_df = pd.DataFrame([
    {
        'content': os.path.basename(r['content_path']),
        'style': os.path.basename(r['style_path']),
        'model': r['model_name'],
        'judge_van_gogh_prob': r['judge_van_gogh_prob'],
        'judge_prediction': r['judge_prediction']
    }
    for r in all_results
])

# Add classifier evaluations if available
for r in all_results:
    if 'VGG19_classifier_van_gogh_prob' in r:
        idx = results_df[results_df['content'] == os.path.basename(r['content_path'])].index[0]
        results_df.loc[idx, 'VGG19_classifier_van_gogh_prob'] = r.get('VGG19_classifier_van_gogh_prob', None)
    if 'AlexNet_classifier_van_gogh_prob' in r:
        idx = results_df[results_df['content'] == os.path.basename(r['content_path'])].index[0]
        results_df.loc[idx, 'AlexNet_classifier_van_gogh_prob'] = r.get('AlexNet_classifier_van_gogh_prob', None)

print("\nResults Summary:")
print(results_df.head(10))

# Statistics by model
print("\n" + "="*60)
print("STATISTICS BY STYLE TRANSFER MODEL")
print("="*60)

for model in ['VGG19', 'AlexNet']:
    model_results = results_df[results_df['model'] == model]
    if len(model_results) > 0:
        print(f"\n{model} Style Transfer:")
        print(f"  Number of images: {len(model_results)}")
        print(f"  Mean Van Gogh probability (Judge): {model_results['judge_van_gogh_prob'].mean():.4f}")
        print(f"  Std Van Gogh probability (Judge): {model_results['judge_van_gogh_prob'].std():.4f}")
        print(f"  Predicted as Van Gogh: {model_results['judge_prediction'].sum()}/{len(model_results)}")
        
        if 'VGG19_classifier_van_gogh_prob' in model_results.columns:
            vgg_probs = model_results['VGG19_classifier_van_gogh_prob'].dropna()
            if len(vgg_probs) > 0:
                print(f"  Mean Van Gogh probability (VGG-19 Classifier): {vgg_probs.mean():.4f}")
        
        if 'AlexNet_classifier_van_gogh_prob' in model_results.columns:
            alexnet_probs = model_results['AlexNet_classifier_van_gogh_prob'].dropna()
            if len(alexnet_probs) > 0:
                print(f"  Mean Van Gogh probability (AlexNet Classifier): {alexnet_probs.mean():.4f}")

# Comparison between VGG-19 and AlexNet
print("\n" + "="*60)
print("VGG-19 vs ALEXNET COMPARISON")
print("="*60)

vgg19_mean = results_df[results_df['model'] == 'VGG19']['judge_van_gogh_prob'].mean()
alexnet_mean = results_df[results_df['model'] == 'AlexNet']['judge_van_gogh_prob'].mean()

print(f"\nVGG-19 mean Van Gogh probability: {vgg19_mean:.4f}")
print(f"AlexNet mean Van Gogh probability: {alexnet_mean:.4f}")
print(f"Difference: {abs(vgg19_mean - alexnet_mean):.4f}")

if vgg19_mean > alexnet_mean:
    print("\nVGG-19 produces more 'Van Gogh-like' results according to the classifier")
else:
    print("\nAlexNet produces more 'Van Gogh-like' results according to the classifier")

# Save results
results_df.to_csv('/kaggle/working/style_transfer_results.csv', index=False)
print("\nResults saved to 'style_transfer_results.csv'")


In [ ]:
# Visualize some example results
print("\n" + "="*60)
print("EXAMPLE STYLIZED IMAGES")
print("="*60)

# Show top 6 results by Van Gogh probability
top_results = sorted(all_results, key=lambda x: x['judge_van_gogh_prob'], reverse=True)[:6]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Top 6 Stylized Images (by Van Gogh Probability)', fontsize=16, fontweight='bold')

for idx, result in enumerate(top_results):
    row = idx // 3
    col = idx % 3
    
    # Display image
    image = result['stylized_image'].cpu().clone().detach().squeeze(0)
    inv_normalize = transforms.Normalize(
        mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
        std=[1/0.229, 1/0.224, 1/0.225]
    )
    image = inv_normalize(image)
    image = transforms.ToPILImage()(image.clamp(0, 1))
    
    axes[row, col].imshow(image)
    title = f"{result['model_name']}\nProb: {result['judge_van_gogh_prob']:.3f}"
    axes[row, col].set_title(title, fontsize=10)
    axes[row, col].axis('off')

plt.tight_layout()
plt.savefig('/kaggle/working/top_stylized_images.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: top_stylized_images.png")


In [ ]:
# Final summary
print("="*60)
print("PART B - FINAL SUMMARY")
print("="*60)

print("\nPart 2B: Hyperparameter Search")
print(f"  Optimal content_weight: {best_style_transfer_params.get('content_weight', 'N/A')}")
print(f"  Optimal style_weight: {best_style_transfer_params.get('style_weight', 'N/A')}")
if 'study' in globals():
    print(f"  Best Van Gogh probability achieved: {study.best_value:.4f}")

print("\nPart 2C: Application & Evaluation")
if 'all_results' in globals():
    print(f"  Total style transfers completed: {len(all_results)}")
if 'num_content' in globals():
    print(f"  Content images used: {num_content}")
if 'num_styles' in globals():
    print(f"  Style images used: {num_styles}")
if 'CONSISTENT_NUM_STEPS' in globals():
    print(f"  Optimization steps per image: {CONSISTENT_NUM_STEPS}")

print("\nModel Comparison:")
if 'vgg19_mean' in globals() and 'alexnet_mean' in globals():
    print(f"  VGG-19 mean probability: {vgg19_mean:.4f}")
    print(f"  AlexNet mean probability: {alexnet_mean:.4f}")
else:
    print("  Run results analysis cell to see comparison")

print("\nSaved Files:")
print("  - best_style_transfer_params.pkl")
print("  - style_transfer_results.csv")
print("  - top_stylized_images.png")
print("  - stylized_images/ (directory with all stylized images)")

print("\nPart B Complete!")
print("="*60)
